In [1]:
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader
from model import Network
from data import ChristmasImages
import model
from torchvision.models import resnet50
import os
import data

In [2]:

def train_model(model, train_loader):
    model.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    running_loss = 0.0

    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    return running_loss / len(train_loader)


In [3]:
def evaluate_model(model, val_loader):
    model.eval()
    predictions = []

    with torch.no_grad():
        for images, ids in val_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())

    return predictions

In [4]:
train_path = "C:\\Users\\Pragya Mahajan\\Documents\\DL assignments\\DL challenge\\data\\train"
test_path = "C:\\Users\\Pragya Mahajan\\Documents\\DL assignments\\DL challenge\\data\\val"


In [6]:
 # Create datasets
train_dataset = ChristmasImages(train_path, training=True)
test_dataset = ChristmasImages(test_path, training=False)

In [8]:
# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)


In [9]:
# Create model instance
model = Network()

In [10]:

# Train the model
for epoch in range(5):  # 5 epochs for demonstration
    train_loss = train_model(model, train_loader)
    print(f'Epoch [{epoch+1}/5], Train Loss: {train_loss:.4f}')

Epoch [1/5], Train Loss: 1.9963
Epoch [2/5], Train Loss: 1.8119
Epoch [3/5], Train Loss: 1.7647
Epoch [4/5], Train Loss: 1.6886
Epoch [5/5], Train Loss: 1.5930


In [11]:

    # Evaluate the model
    predictions = evaluate_model(model, test_loader)


In [12]:

    # Save predictions to CSV
    df = pd.DataFrame({'Id': test_dataset.image_files, 'Category': predictions})
    df['Id'] = df['Id'].apply(lambda x: int(x.split('.')[0]))  # Extracting image ID
    df.to_csv('predictions.csv', index=False)

    # Save the trained model
    model.save_model()
